Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [436]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os # use this to access your environment variables
import requests # this will be used to call the APIs
from unidecode import unidecode # remove accents from venues e.g. café --> cafe
import re # regex search for strings


FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')
YELP_KEY = os.getenv('YELP_API_KEY')

In [161]:
bike_stations = pd.read_csv("../data/bike_stations_sat_3pm_clean.csv")

**This bike_stations data is based on the Saturday, 3pm citybikes API call**

In [163]:
bike_stations

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,18,1,94.74,"43.665269,-79.319796"
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,1,14,6.67,"43.67142,-79.445947"
2,a09c67c0b419654d907c9134b108e328,Queen St E / Rhodes Ave,43.666224,-79.317693,12,11,52.17,"43.666224,-79.317693"
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,5,32,13.51,"43.653236,-79.376716"
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,3,32,8.57,"43.663722,-79.380288"
...,...,...,...,...,...,...,...,...
821,9be5f078a1ed47fc11cd3cee45260f63,Kennedy Rd/Ranstone Gdns (Jack Goodlad Park),43.741906,-79.271819,1,10,9.09,"43.741906,-79.271819"
822,4ae37f3bddfb819954a15143d277dbd9,Eglinton Ave E / Brimley Rd,43.736953,-79.247984,8,11,42.11,"43.736953,-79.247984"
823,e7968ab22d9a15db0673f463144428eb,College Park South,43.659457,-79.382365,7,11,38.89,"43.659457,-79.382365"
824,62acc308c0f93ff09d28e06c73afc3ec,165 McRae Dr,43.705875,-79.368006,4,11,26.67,"43.705875,-79.368006"


In [104]:
# testing the ability to parse for POI category, number of POIs for a sample API response, need the non-cleaned version
venue_df = pd.read_csv("../data/test_venue_df.csv")

In [286]:
sample_bike_df = bike_stations.sample(n=2)

In [288]:
sample_bike_df

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll
76,c497911a9c04bebbbc32e5211c081f85,Bridgman Ave / Kendal Ave,43.675556,-79.410278,6,9,40.00,"43.675556,-79.410278"
218,a7cdbedefded6e117505bde22c24ef84,Western Battery Rd / Pirandello St,43.640075,-79.414156,1,30,3.23,"43.640075,-79.414156"


**Strategy is as follows:
Steps for the 10 stations in sample_bike_df, ensure it works for the 820 rows in the whole city bike dataset**:

1. Define my get_venues_fs function to call the Foursquare API. Set it up so that it can take a concatenated ll string, like the ll column of bike_stations and sample_bike_df.
2. Initialize a new bike_stations df with the columns to fill in for # of POIs, and # of POIs in certain categories.
3. Loop through each bike station, make the get_venues_fs call with its ll column to find venue results with the data in step 2. 
4. With the returned results, we'll parse for the number of POI categories, and the number of POIs generally (know that the upper limit of returns is 50, radius of the call is 800m to make <=50 results meaningful)
5. We should have a dataframe returned containing 1) the original ll call which we can use to join the bike_stations table later if we wish, 2) the number of POIs and number of establishments within POI categories, to train the model

Notes:

**I am doing step 2 here and not in yelp_foursquare_EDA to avoid bringing city_bikes data into the yelp_foursquare notebook, so any cross-referencing of city bikes with venue data will happen here**.

**In the likely event that some bike station ll's will retrieve establishments that were retrieved by another bike station (e.g. in Downtown Toronto where several stations will be within 800m of each other), I am retrieving the fsq_id for the establishments which will be checked for dupes, with dupes removed, before removing the column in cleaning to join it with a cleaned bike_stations df**

In [461]:
# Step 1 from Strategy: Defining the foursquare API Call here

def get_venues_fs(ll, radius, api_key, categories, limit):
    """
    Get amenities and POIs from Foursquare API call
    Args:
        ll (str): concatenated latitude,longitude that is used to supply the API and return POIs
        radius: number marking the radius in metres for the POIs to be collected from
        api_key (str): foursquare API to use for query (imported above)
        categories (str) : Foursquare-recognized place type. four categories provided at bottom of this cell
        limit (int): will be given as 50, the maximum for this API
    
    Returns:
        response: response object from the requests library.
    """
    url = "https://api.foursquare.com/v3/places/search"
    
    headers = {
        "Accept": "application/json",
        "Authorization": api_key
    }
    
    params = {
        "ll": ll,
        "radius": radius,
        "categories": categories,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200: # 200 is success
        return response.json()
    else:
        response.raise_for_status()

categories = '10035,13003,13065,16000' # Category codes - bars, restaurants, live shows, outdoors


In [463]:
# Step 2: get the bike station dataframe ready for the Foursquare API data to fill in in step 3

sample_bike_df['n_pois'] = 0
sample_bike_df['n_bar_restaurant'] = 0
sample_bike_df['n_cafe'] = 0
sample_bike_df['n_live'] = 0
sample_bike_df['n_park'] = 0
sample_bike_df

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll,n_pois,n_bar_restaurant,n_cafe,n_live,n_park
76,c497911a9c04bebbbc32e5211c081f85,Bridgman Ave / Kendal Ave,43.675556,-79.410278,6,9,40.00,"43.675556,-79.410278",0,0,0,0,0
218,a7cdbedefded6e117505bde22c24ef84,Western Battery Rd / Pirandello St,43.640075,-79.414156,1,30,3.23,"43.640075,-79.414156",0,0,0,0,0


In [459]:
# Step 3:

# lists for the augmented bike_stations for the model - number of POIs, bars/restaurants/live venues, parks to populate columns for the model training
n_poi_list = [] # this is solved by n_poi_list.append(len(res['results']))
n_restobar_list = []
n_live_list = []
n_park_list = []
n_cafe_list = []

# lists for the venues returned via the API call

fsq_id_list = []
ll_list = [] # this is here for the ll from the bike station that 'summoned' this venue. For joining the SQL tables.
venue_ll_list = [] # this is the ll for the venue itself.
name_list = []
address_list = []
category_list = []

# set up regex patterns for finding # of bar/restaurant, cafes, parks, live venues to search within the category column. These are loose 'contain' matches
bar_restaurant_pattern = re.compile(r'\b(bar|restaurant|gastropub|lounge|bbq|pub|grill|burger|chicken|diner|Fish and Chips|pizzeria|tavern|night club|bistro|steakhouse|nightclub)\b', re.IGNORECASE)
live_venue_pattern = re.compile(r'\b(concert|music venue|music|comedy|live|theater)\b', re.IGNORECASE)
park_pattern = re.compile(r'\b(park|playground|monument|plaza|structure|Historic and Protected Site|Lake|Landmarks and Outdoors|monument|garden)\b', re.IGNORECASE)
cafe_pattern = re.compile(r'\b(cafe|coffee|coffee shop|tea|bakery|donut|deli|dessert|bagel|sandwich)\b', re.IGNORECASE)


In [465]:
# Step 3 and Step 4:

for index, row in sample_bike_df.iterrows():
    n_restobar = 0 # I will iterate over each venue returned for the bike station's coordinates and ticker up for a positive match to the regex
    n_live = 0
    n_park = 0
    n_cafe = 0
    lat_long = row['ll']
    
    res = get_venues_fs(ll=lat_long, radius=800, api_key=FOURSQUARE_KEY, categories=categories, limit=50)
    
    n_poi_list.append(len(res['results'])) # the number of POIs is simply length of the results, no need to ticker.

    # step 4 - populating the df with POIs
    
    for venue in res['results']:
        fsq_id_list.append(venue['fsq_id'])
        ll_list.append(lat_long)
        venue_ll_list.append(f"{venue['geocodes']['main']['latitude']},{venue['geocodes']['main']['longitude']}")
        name_list.append(venue['name'])
        address_list.append(venue['location'].get('address', 'Address not available'))
        category_list.append(venue['categories'][0]['name'])
        
        category_name = venue['categories'][0]['name'] # capturing the first category name in order to regex check it for one of our major categories of interest
        category_name = unidecode(category_name)
        
        if bar_restaurant_pattern.search(category_name):
            n_restobar += 1
        elif live_venue_pattern.search(category_name):
            n_live += 1
        elif park_pattern.search(category_name):
            n_park += 1
        elif cafe_pattern.search(category_name):
            n_cafe += 1

    n_restobar_list.append(n_restobar)
    n_live_list.append(n_live)
    n_park_list.append(n_park)
    n_cafe_list.append(n_cafe)
    

In [467]:
print(n_poi_list)
print(n_restobar_list)
print(n_live_list)
print(n_park_list)
print(n_cafe_list)

#print(f"{row['name']}, {row['ll']}, {row['n_pois']}") - just a spare print for the cell above

[50, 50]
[21, 31]
[2, 1]
[14, 9]
[11, 8]


In [469]:
sample_bike_df['n_pois'] = n_poi_list
sample_bike_df['n_bar_restaurant'] = n_restobar_list
sample_bike_df['n_live'] = n_live_list
sample_bike_df['n_park'] = n_park_list
sample_bike_df['n_cafe'] = n_cafe_list

In [471]:
sample_bike_df

,id,name,latitude,longitude,free_bikes,empty_slots,bike_availability,ll,n_pois,n_bar_restaurant,n_cafe,n_live,n_park
76,c497911a9c04bebbbc32e5211c081f85,Bridgman Ave / Kendal Ave,43.675556,-79.410278,6,9,40.00,"43.675556,-79.410278",50,21,11,2,14
218,a7cdbedefded6e117505bde22c24ef84,Western Battery Rd / Pirandello St,43.640075,-79.414156,1,30,3.23,"43.640075,-79.414156",50,31,8,1,9


I noticed that for 50 POIs the sum of restobar, cafe, live, park was sometimes only coming to about 45 or 40, I went back to check which other tags were not being picked up by my Regex statements:
- Structure, e.g. the famous Casa Loma building in Toronto
- Lake
- Historic and Protected Site
- Landmarks and Outdoors

I noticed others such as Tree Service and Bridge but I judged via the names and checking on Google maps that these are not necessarily points of interest and may be more related to key infrastructure and maintenance, which probably does not influence the bike station availability. So, whilst I captured some extra key categories missed in my initial regex by taking random bike station samples, I am happy with not every single POI under the Foursquare categories being captured by my regex statements.

In [475]:
# set up a dataframe for all the venues that we collected from the Foursquare API Call

venues_dict = {
    'foursquare_id': fsq_id_list,
    'll': ll_list,
    'venue_ll': venue_ll_list,
    'name': name_list,
    'address': address_list,
    'category': category_list
}

#print(fsq_id_list, ll_list, name_list, address_list, category_list)

sample_venues_df = pd.DataFrame(venues_dict)
sample_venues_df


,foursquare_id,ll,venue_ll,name,address,category
0,4c62c59ce1621b8dd0332453,"43.675556,-79.410278","43.674776,-79.408277",Roti Cuisine of India,308 Dupont St,South Indian Restaurant
1,4bef48fcc80dc9284ec827e3,"43.675556,-79.410278","43.6781,-79.40939",Casa Loma,1 Austin Terr,Structure
2,54400bee498e61defb21f0a5,"43.675556,-79.410278","43.673854,-79.411685",Schmaltz Appetizing,414 Dupont St,Deli
3,4d8e7fde1716a14396a634f7,"43.675556,-79.410278","43.677709,-79.408091",Baldwin Steps,Davenport Rd,Historic and Protected Site
4,4ad4c062f964a520c0f720e3,"43.675556,-79.410278","43.674897,-79.412787",Tarragon Theatre,30 Bridgman Ave,Music Venue
...,...,...,...,...,...,...
95,4bd3294acaff9521e125d4f0,"43.640075,-79.414156","43.640903,-79.410113",Garrison Creek Community Garden,Address not available,Garden
96,4bd835dbf645c9b6cf59a7e0,"43.640075,-79.414156","43.641029,-79.418406",The King Street CN Viaduct,1050 King St W,Bridge
97,5f639b6e628ebe72db204974,"43.640075,-79.414156","43.638158,-79.41776",Chiang Mai Thai Kitchen & Bar,171 E Liberty St,Thai Restaurant
98,4b673d73f964a5203d432be3,"43.640075,-79.414156","43.63839,-79.406861",Garrison Commons,100 Garrison Rd,Park


In [ ]:
# Json normalize!!!

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.